In [ ]:
import os
# Find the latest version of spark 3.0 from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.3'
spark_version = 'spark-3.2.1'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Get:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Hit:2 http://archive.ubuntu.com/ubuntu bionic InRelease
Hit:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Get:4 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Ign:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:6 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:7 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Get:8 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:9 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Hit:10 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:11 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Hit:12 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease
Get:14 http://archive.ubuntu.com/ubuntu b

In [ ]:
# Download the Postgres driver that will allow Spark to interact with Postgres.
!wget https://jdbc.postgresql.org/download/postgresql-42.2.16.jar

--2022-06-24 05:02:32--  https://jdbc.postgresql.org/download/postgresql-42.2.16.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1002883 (979K) [application/java-archive]
Saving to: ‘postgresql-42.2.16.jar’

postgresql-42.2.16. 100%[===================>] 979.38K  --.-KB/s    in 0.08s   

2022-06-24 05:02:32 (11.3 MB/s) - ‘postgresql-42.2.16.jar’ saved [1002883/1002883]



In [ ]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("US_Pollution_2000_2016").config("spark.driver.extraClassPath","/content/postgresql-42.2.16.jar").getOrCreate()

In [ ]:
from pyspark import SparkFiles
url = "https://pollution-sight-data.s3.us-west-1.amazonaws.com/updated_us_pollution_2000_2016.csv"
spark.sparkContext.addFile(url)
pollution_df = spark.read.csv(SparkFiles.get("updated_us_pollution_2000_2016.csv"), header=True, inferSchema=True)
pollution_df.show()

+-------+--------+-------+----------+---------+-----------+----------+------+--------+----------+---------+-----+---------+-----------+----------+------+--------+----------+---------+-----+
|usstate|  county|   city|local_date|   no2avg|no2maxvalue|no2maxhour|no2aqi|   o3avg|o3maxvalue|o3maxhour|o3aqi|   so2avg|so2maxvalue|so2maxhour|so2aqi|   coavg|comaxvalue|comaxhour|coaqi|
+-------+--------+-------+----------+---------+-----------+----------+------+--------+----------+---------+-----+---------+-----------+----------+------+--------+----------+---------+-----+
|Arizona|Maricopa|Phoenix|2000-01-01|19.041667|       49.0|        19|    46|  0.0225|      0.04|       10|   34|      3.0|        9.0|        21|  13.0|0.878947|       2.2|       23| 25.0|
|Arizona|Maricopa|Phoenix|2000-01-02|22.958333|       36.0|        19|    34|0.013375|     0.032|       10|   27| 1.958333|        3.0|        22|   4.0|1.066667|       2.3|        0| 26.0|
|Arizona|Maricopa|Phoenix|2000-01-03|   38.125|   

In [ ]:
pollution_df.printSchema()

root
 |-- usstate: string (nullable = true)
 |-- county: string (nullable = true)
 |-- city: string (nullable = true)
 |-- local_date: string (nullable = true)
 |-- no2avg: double (nullable = true)
 |-- no2maxvalue: double (nullable = true)
 |-- no2maxhour: integer (nullable = true)
 |-- no2aqi: integer (nullable = true)
 |-- o3avg: double (nullable = true)
 |-- o3maxvalue: double (nullable = true)
 |-- o3maxhour: integer (nullable = true)
 |-- o3aqi: integer (nullable = true)
 |-- so2avg: double (nullable = true)
 |-- so2maxvalue: double (nullable = true)
 |-- so2maxhour: integer (nullable = true)
 |-- so2aqi: double (nullable = true)
 |-- coavg: double (nullable = true)
 |-- comaxvalue: double (nullable = true)
 |-- comaxhour: integer (nullable = true)
 |-- coaqi: double (nullable = true)



In [ ]:
# Convert local_date column to date
from pyspark.sql.functions import to_date 
pollution_df = pollution_df.select(["usstate", "county", "city", to_date("local_date", "yyyy-MM-dd").alias("local_date"), "no2avg", "no2maxvalue", "no2maxhour", "no2aqi", "o3avg", "o3maxvalue", "o3maxhour", "o3aqi", "so2avg", "so2maxvalue", "so2maxhour", "so2aqi", "coavg", "comaxvalue", "comaxhour", "coaqi"])
pollution_df.show()

+-------+--------+-------+----------+---------+-----------+----------+------+--------+----------+---------+-----+---------+-----------+----------+------+--------+----------+---------+-----+
|usstate|  county|   city|local_date|   no2avg|no2maxvalue|no2maxhour|no2aqi|   o3avg|o3maxvalue|o3maxhour|o3aqi|   so2avg|so2maxvalue|so2maxhour|so2aqi|   coavg|comaxvalue|comaxhour|coaqi|
+-------+--------+-------+----------+---------+-----------+----------+------+--------+----------+---------+-----+---------+-----------+----------+------+--------+----------+---------+-----+
|Arizona|Maricopa|Phoenix|2000-01-01|19.041667|       49.0|        19|    46|  0.0225|      0.04|       10|   34|      3.0|        9.0|        21|  13.0|0.878947|       2.2|       23| 25.0|
|Arizona|Maricopa|Phoenix|2000-01-02|22.958333|       36.0|        19|    34|0.013375|     0.032|       10|   27| 1.958333|        3.0|        22|   4.0|1.066667|       2.3|        0| 26.0|
|Arizona|Maricopa|Phoenix|2000-01-03|   38.125|   

In [ ]:
pollution_df.printSchema()

root
 |-- usstate: string (nullable = true)
 |-- county: string (nullable = true)
 |-- city: string (nullable = true)
 |-- local_date: date (nullable = true)
 |-- no2avg: double (nullable = true)
 |-- no2maxvalue: double (nullable = true)
 |-- no2maxhour: integer (nullable = true)
 |-- no2aqi: integer (nullable = true)
 |-- o3avg: double (nullable = true)
 |-- o3maxvalue: double (nullable = true)
 |-- o3maxhour: integer (nullable = true)
 |-- o3aqi: integer (nullable = true)
 |-- so2avg: double (nullable = true)
 |-- so2maxvalue: double (nullable = true)
 |-- so2maxhour: integer (nullable = true)
 |-- so2aqi: double (nullable = true)
 |-- coavg: double (nullable = true)
 |-- comaxvalue: double (nullable = true)
 |-- comaxhour: integer (nullable = true)
 |-- coaqi: double (nullable = true)



In [ ]:
# Connect to AWS RDS instance
# Store environmental variable
from getpass import getpass
password = getpass('Enter database password')

# Configure settings for RDS
mode = "append"
jdbc_url="jdbc:postgresql://<connection_string>:5432/<database_name>"
config = {"user":"postgres", 
          "password": password, 
          "driver":"org.postgresql.Driver"}

Enter database password··········


In [ ]:
# Write pollution_df to table in RDS
pollution_df.write.jdbc(url=jdbc_url, table='us_pollution_2000_2016', mode=mode, properties=config)